# Present the symbolic derivation of existing models

In [ ]:
%matplotlib widget
import os, importlib, inspect
from IPython.display import display, Math, Latex, Markdown
from bmcs_matmod.gsm.gsm_models.gsm_base import GSMBase, sp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
modules = []

# Collect all relevant modules and their classes
for file_name in os.listdir('.'):
    if file_name.endswith('.py') and file_name not in ['gsm_base.py', '__init__.py']:
        module_path = f"bmcs_matmod.gsm.gsm_models.{file_name[:-3]}"
        module = importlib.import_module(module_path)
        for name, obj in inspect.getmembers(module, inspect.isclass):
            if issubclass(obj, GSMBase) and obj is not GSMBase:
                modules.append((name, obj))

# Sort the modules by class names
modules.sort(key=lambda x: x[0])

gsm_models = {name: obj() for name, obj in modules}
gsm_models

In [ ]:
t_, eps_max_, t_1_, t_2_, t_3_ = sp.symbols('t eps_max t_1 t_2 t_3', real=True, positive=True)
eps_t_ = sp.Piecewise(
    (eps_max_ / t_1_ * t_, t_ < t_1_),
    (eps_max_, t_ < t_2_),
    (eps_max_ - eps_max_/(t_3_-t_2_)*(t_ - t_2_), t_ < t_3_),
    (0, True)
)
get_eps_t = sp.lambdify((t_, eps_max_, t_1_, t_2_, t_3_), eps_t_, 'numpy')
final_time = 1
eps_max = 1.5
t_t = np.linspace(0, final_time, 2000)
sig_n_t = get_eps_t(t_t, eps_max, 0.25*final_time, 0.5*final_time, 0.75*final_time)
fig, ax = plt.subplots(1,1)
ax.plot(t_t, sig_n_t)

In [ ]:
mparams = dict(E = 20, K = 20, S = 1, c = 1, r = 1, eps_0 = 0, f_c = 15, eta_vp=50, eta_ve=500)
response = {}
selection = [] # ['GSM1D_VEVP', 'GSM1D_VEVPD']
if len(selection) == 0:
    selection = gsm_models.keys()
for name in selection:
    gsm = gsm_models[name]
    print(f'name: {name}')
    response[name] = gsm.get_response(sig_n_t, t_t, **mparams)


In [ ]:
fig, (ax_sig, ax_eps) = plt.subplots(1, 2, figsize=(12, 5), gridspec_kw={'width_ratios': [7, 3]})
ax_iter = ax_eps.twinx()
ax_iter.set_ylabel('Iterations')
# ax_eps.plot(t_t, eps_n_t, color='black')
# ax_eps.set_xlabel(r'$t$')
# ax_eps.set_ylabel(r'$\varepsilon$')
for name, resp in response.items():
    t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_tI, lam_t, (d_t_t, d_eps_ta) = resp
    eps_t = eps_ta[:, 0]
    iter_t = iter_tI[:, 0]
    sig_t = sig_ta[:, 0, 0]
    ax_sig.plot(eps_t, sig_t, label=rf'{name}')
    ax_iter.plot(t_t, iter_t, label=rf'{name}', linestyle='dashed')
    # Add a box with the material parameters
    param_text = ', '.join([f'{key}: {value}' for key, value in mparams.items()])
    param_text = '\n'.join([', '.join(param_text.split(', ')[i:i+5]) for i in range(0, len(param_text.split(', ')), 5)])
    props = dict(boxstyle='round', facecolor='lightgray', edgecolor='lightgray', alpha=0.5)
    ax_sig.text(0.5, 0.97, param_text, transform=ax_sig.transAxes, fontsize=11,
                verticalalignment='top', horizontalalignment='center', bbox=props)
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend(fontsize='small')
fig.tight_layout()

In [ ]:
mparams = dict(E = 20, K = 20, S = 1, c = 1, r = 1, eps_0 = 0, f_c = 15, 
               eta_vp=500, eta_ve=500)
response = {}
selection = ['GSM1D_EVPD', 'GSM1D_VEVPD']
if len(selection) == 0:
    selection = gsm_models.keys()
for name in selection:
    gsm = gsm_models[name]
    print(f'name: {name}')
    response[name] = gsm.get_response(sig_n_t, t_t, **mparams)


In [ ]:
fig, (ax_sig, ax_eps) = plt.subplots(1, 2, figsize=(12, 5), gridspec_kw={'width_ratios': [7, 3]})
ax_iter = ax_eps.twinx()
ax_iter.set_ylabel('Iterations')
# ax_eps.plot(t_t, eps_n_t, color='black')
# ax_eps.set_xlabel(r'$t$')
# ax_eps.set_ylabel(r'$\varepsilon$')
for name, resp in response.items():
    t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_tI, lam_t, (d_t_t, d_eps_ta) = resp
    eps_t = eps_ta[:, 0]
    iter_t = iter_tI[:, 0]
    sig_t = sig_ta[:, 0, 0]
    ax_sig.plot(eps_t, sig_t, label=rf'{name}')
    ax_iter.plot(t_t, iter_t, label=rf'{name}', linestyle='dashed')
    # Add a box with the material parameters
    param_text = ', '.join([f'{key}: {value}' for key, value in mparams.items()])
    param_text = '\n'.join([', '.join(param_text.split(', ')[i:i+5]) for i in range(0, len(param_text.split(', ')), 5)])
    props = dict(boxstyle='round', facecolor='lightgray', edgecolor='lightgray', alpha=0.5)
    ax_sig.text(0.5, 0.97, param_text, transform=ax_sig.transAxes, fontsize=11,
                verticalalignment='top', horizontalalignment='center', bbox=props)
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend(fontsize='small')
fig.tight_layout()